In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from  datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime as datetime
from sklearn.metrics import accuracy_score
from category_encoders.target_encoder import TargetEncoder
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")


In [ ]:
train.head(100)


In [ ]:
train.tail()


In [ ]:
train.columns


In [ ]:
train.describe()


In [ ]:
# print("Mean",np.mean(train))
# print("Standard Deviation", np.std(train))
# print("Min", np.min(train))
# print("Max", np.max(train))


In [ ]:
train['Cover_Type'].describe()


In [ ]:
train.groupby('Cover_Type').size()


In [ ]:
plt.hist(train['Cover_Type'])


In [ ]:
#skewness and kurtosis
print("Skewness: %f" % train['Cover_Type'].skew())
print("Kurtosis: %f" % train['Cover_Type'].kurt())


In [ ]:
var = 'Hillshade_Noon'
data = pd.concat([train['Cover_Type'], train[var]], axis=1)
data.plot.scatter(x=var, y='Cover_Type', ylim=(0,800000))


In [ ]:
#most correlated 10 columns
corr=train.corr()
v=10
colmn = corr.nlargest(v, 'Cover_Type')
colmn


In [ ]:
colmn = corr.nlargest(v, 'Cover_Type')['Cover_Type'].index 
xm = np.corrcoef(train[colmn].values.T)
sns.set(font_scale=1.25)
plt.figure(figsize=(18, 18))
hm = sns.heatmap(xm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},
                 yticklabels=colmn.values, xticklabels=colmn.values)
plt.show()


In [ ]:
total= train.isnull().sum().sort_values(ascending=False)
total


In [ ]:
# drop ID
train_X = train.drop('Id', axis=1)
#drop output
train_X = train_X.drop('Cover_Type', axis=1)
#contain output on y container
train_y = train['Cover_Type']


In [ ]:
train_y.head()


In [ ]:
#trin_test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.22, random_state=2021 )


In [ ]:
del train, train_X, train_y


In [ ]:
y_test


In [ ]:
#Encoding those data
nums_cols = [col for col in X_train.columns if X_train[col].dtype in ["float16","float32","float64"]]
catgo_cols = [col for col in X_train.columns if X_train[col].dtype not in ["float16","float32","float64"]]


In [ ]:
test=test.drop('Id', axis=1)


In [ ]:
d_test = test

for cols in catgo_cols:
    enc = TargetEncoder(cols=[cols])
    x = enc.fit_transform(X_train, y_train)
    y= enc.transform(X_test)
    z = enc.transform(d_test)


In [ ]:
del test


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_train)
train_X = pd.DataFrame(scaler.transform(x))
test_X = pd.DataFrame(scaler.transform(y))
test = pd.DataFrame(scaler.transform(z))


In [ ]:
del x,y,z


In [ ]:
train_X


In [ ]:
train_X = train_X.to_numpy().astype(np.float32)
y_train =y_train.to_numpy().astype(np.float32)
y_test =y_test.to_numpy().astype(np.float32)
test_X =test_X.to_numpy().astype(np.float32)
test =test.to_numpy().astype(np.float32)


In [ ]:
# XGBClassifier = xgb.XGBClassifier(max_depth = 9,
#                                  learning_rate = 0.008,
#                                  n_estimators = 20000,
# #                                  objective = 'binary:logistic',
#                                  objective = 'multi:softmax',
#                                  tree_method = 'gpu_hist',
#                                  booster = 'gbtree',
#                                  gamma = 0.75,
#                                  max_delta_step = 6,
#                                  min_child_weight = 9,
#                                  subsample = 0.8326,
#                                  colsample_bytree = 0.7,
#                                  colsample_bynode= 0.7523,
#                                  colsample_bylevel=0.8231,
#                                  predictor= 'gpu_predictor'
# #                                  n_jobs = -1
#                                  )


In [ ]:
from xgboost import XGBClassifier


# declare parameters
params = {
#             'objective':'binary:logistic',/
            'objective' : 'multi:softmax',
            'tree_method': 'gpu_hist',
            'eval_metric': 'mlogloss',
            'booster' : 'gbtree',
            'gamma' : 0.75,
            'max_depth': 7,
            'alpha': 10,
            'learning_rate': .007,
            'n_estimators':2000,
            'predictor': 'gpu_predictor'
        }         
           
          
# instantiate the classifier 
xgb = XGBClassifier(**params)
xgb.fit(train_X, y_train,
          early_stopping_rounds=200,
          eval_set=[(test_X,y_test)],
          verbose=True)


In [ ]:
preds_valid = xgb.predict(test_X).astype('int')
acc = accuracy_score(y_test,  preds_valid)
print("accuracy score:", acc)


In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
sub['Cover_Type'] =xgb.predict(test).astype('int')
sub.to_csv("submission.csv",index=False)
sub.head()
